In [ ]:
#　决策树预测泰坦尼克号生还

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [39]:
data = pd.read_csv('./train.csv', index_col=0)
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [41]:
# 删除姓名,票号,货仓三列
data.drop(columns=['Name', 'Cabin', 'Ticket'], inplace=True)

In [42]:
data.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [43]:
# 将性别做编码
d = {'female':0,'male':1}
data.Sex = data.Sex.map(d)

In [44]:
data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [45]:
# 删除Embarked空值行
data = data.dropna(axis=0, subset=['Embarked'])
# 对Embarked进行编码
d = {'S':0, 'C':1, 'Q':2}
data.Embarked = data.Embarked.map(d)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 1 to 891
Data columns (total 8 columns):
Survived    889 non-null int64
Pclass      889 non-null int64
Sex         889 non-null int64
Age         712 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
Embarked    889 non-null int64
dtypes: float64(2), int64(6)
memory usage: 62.5 KB


In [47]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=666)

In [48]:
X_train.Age.fillna(X_train.Age.mean() , inplace= True)
X_test.Age.fillna(X_train.Age.mean() , inplace= True)

E:\Anaconda\Anaconda\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 622 entries, 618 to 238
Data columns (total 7 columns):
Pclass      622 non-null int64
Sex         622 non-null int64
Age         622 non-null float64
SibSp       622 non-null int64
Parch       622 non-null int64
Fare        622 non-null float64
Embarked    622 non-null int64
dtypes: float64(2), int64(5)
memory usage: 38.9 KB


In [50]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 488 to 671
Data columns (total 7 columns):
Pclass      267 non-null int64
Sex         267 non-null int64
Age         267 non-null float64
SibSp       267 non-null int64
Parch       267 non-null int64
Fare        267 non-null float64
Embarked    267 non-null int64
dtypes: float64(2), int64(5)
memory usage: 16.7 KB


In [55]:
# 建模
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

print('训练集: ', dtc.score(X_train, y_train))
print('测试集: ', dtc.score(X_test, y_test))

训练集:  0.9919614147909968
测试集:  0.7490636704119851


In [56]:
#　观测结果可知，训练模型过拟合

In [57]:
# 使用网格搜索调参
from sklearn.model_selection import GridSearchCV

In [90]:
# 确定参数搜索范围
# max_depth : 1-10
# criterion  基尼和熵两个
#   min_samples_leaf  (1 - 50 , 5)
# min_impurity_decrease  (0 - 0.5  , 走20次)

# 构造网格搜索字典,字典的key用参数名称  , 字典的value 用搜索的序列
d = {"criterion":['gini' , "entropy"] , 
    "max_depth": range(1, 3),
    "min_samples_leaf": range(1, 51, 5), 
    "min_impurity_decrease":np.linspace(0, 0.5, 20)}

In [91]:
dtc = DecisionTreeClassifier(random_state=0)
"""
estimator, 评估器: 实例化的算法模型
    param_grid, # 网格搜索参数字典
    scoring=None, # 模型评估的指标, 默认是准确率
    n_jobs=None, # 计算使用的CPU处理器个数
    cv=5 , #  交叉验证次数
    verbose=0, # 调参中的日志输出
""" 
grid = GridSearchCV(dtc, param_grid=d, n_jobs=-1, cv=5, verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 4000 out of 4000 | elapsed:    8.4s finished
E:\Anaconda\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=0,
                                              splitter='best'),
             iid='warn', n_jo...
                         'min_impurity_decrease': array([0.        , 0.02631579, 0.05263158, 0.07894737, 0.10526316,
       0.13157895, 0.157894

In [92]:
# 查看最优参数
grid.best_params_

{'criterion': 'gini',
 'max_depth': 1,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1}

In [93]:
# 查看分数
print('训练集: ', grid.score(X_train, y_train))
print('测试集: ', grid.score(X_test, y_test))

训练集:  0.792604501607717
测试集:  0.7715355805243446


In [94]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=1, min_impurity_decrease=0.0, min_samples_leaf=1)
dtc.fit(X_train, y_train)
print('训练集: ', dtc.score(X_train, y_train))
print('测试集: ', dtc.score(X_test, y_test))

训练集:  0.792604501607717
测试集:  0.7715355805243446
